In [3]:
import os
import shutil
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import errno
import stat

def handle_remove_readonly(func, path, exc):
    exc_value = exc[1]
    if exc_value.errno == errno.ENOENT:
        pass  # 文件不存在，忽略
    elif exc_value.errno == errno.EACCES:
        os.chmod(path, stat.S_IWUSR)  # 修改文件权限
        func(path)  # 再次尝试删除操作
    else:
        raise  # 其他错误，抛出异常

def decimal_to_custom_base(n):
    """将十进制数字转换为使用自定义非混淆字符集的基25字符串。"""
    characters = 'acdefhjkmnpqrtuvwxyz23478'
    base = len(characters)
    if n == 0:
        return characters[0]
    result = ""
    while n > 0:
        n, remainder = divmod(n, base)
        result = characters[remainder] + result
    return result

def human_readable_size(size):
    """将字节转换为更易读的格式。"""
    if size < 1024:
        return f"{size} B"
    elif size < 1024 ** 2:
        return f"{size / 1024:.2f} KB"
    elif size < 1024 ** 3:
        return f"{size / (1024 ** 2):.2f} MB"
    else:
        return f"{size / (1024 ** 3):.2f} GB"

def resize(directory, grayscale=True):
    """调整目录中所有图像的大小为256x256，转换为灰度，并使用基25编码重命名。"""
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp']
    successful_files = set()
    failed_counter = 0

    target_dir = Path(directory)
    thumbnail_dir = target_dir / "thumbnail"
    mapping_file_path = thumbnail_dir / "mapping.txt"

    if not target_dir.is_dir():
        tqdm.write(f"目录不存在: {directory}")
        return

    if thumbnail_dir.exists():
        tqdm.write(f"删除已存在的缩略图目录: {thumbnail_dir}")
        shutil.rmtree(thumbnail_dir, onerror=handle_remove_readonly)
    thumbnail_dir.mkdir(parents=True, exist_ok=True)

    total_size = 0
    image_paths = []
    for root, dirs, files in os.walk(directory):
        for name in files:
            if any(name.lower().endswith(ext) for ext in image_extensions):
                path = Path(root) / name
                image_paths.append(path)

    progress = tqdm(image_paths, desc="正在扫描文件夹")
    images = []
    for path in progress:
        file_size = path.stat().st_size
        total_size += file_size
        images.append(path)
        progress.set_description(f"已收集 {len(images)} 张图片, 总大小: {human_readable_size(total_size)}")

    tqdm.write(f"共找到 {len(images)} 张图片, 总大小: {human_readable_size(total_size)}")

    pbar = tqdm(images, desc="处理图片中")
    mapping_lines = []
    for index, source_path in enumerate(pbar):
        new_name = decimal_to_custom_base(index + 1)  # 基于0的索引
        target_path = thumbnail_dir / f"{new_name}.jpg"
        try:
            if grayscale:
                img = Image.open(source_path).convert('L')
            else:
                img = Image.open(source_path)
            img = img.resize((256, 256), Image.LANCZOS)
            img.save(target_path, format='JPEG', quality=85)
            successful_files.add(new_name + '.jpg')
            mapping_lines.append(f"{source_path}*{new_name}\n")
        except Exception as e:
            pbar.write(f"处理文件错误 {source_path}: {e}")
            failed_counter += 1

    thumbnail_size = sum(p.stat().st_size for p in thumbnail_dir.iterdir())
    tqdm.write(f"处理完成。成功操作: {len(successful_files)}, 失败操作: {failed_counter}")
    tqdm.write(f"缩略图总大小: {human_readable_size(thumbnail_size)}")
    if total_size > 0:
        percentage = (thumbnail_size / total_size) * 100
        tqdm.write(f"缩略图占原图的 {percentage:.2f}% 大小。")

    with open(mapping_file_path, 'w') as file:
        file.writelines(mapping_lines)

In [4]:
# target_directory = "/Volumes/192.168.1.173/pic/今田美樱_939[2123_MB]"
target_directory = "/Users/chenweichu/dev/data/test_副本"

tqdm.write("开始处理...")
resize(target_directory)

开始处理...
删除已存在的缩略图目录: /Users/chenweichu/dev/data/test_副本/thumbnail


已收集 85 张图片, 总大小: 311.83 MB: 100%|██████████| 85/85 [00:00<00:00, 3443.00it/s]


共找到 85 张图片, 总大小: 311.83 MB


处理图片中: 100%|██████████| 85/85 [00:11<00:00,  7.31it/s]

处理完成。成功操作: 85, 失败操作: 0
缩略图总大小: 1020.35 KB
缩略图占原图的 0.32% 大小。
